In [1]:
from selenium import webdriver
import sys
import time
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd


### Amoños
antes de correr debes descargar el chrome driver acorde a tu maquina
https://sites.google.com/chromium.org/driver/downloads


### helper functions

In [2]:
def get_companyname_rating(company_name):
    """gets the company name and (if exist) gets the company rating too"""
    splited = company_name.split("\n")
    #company name comes with rating 
    if len(splited) > 1:
        return splited[0],  splited[1]
    #just company name
    else:
        return company_name, -1


def get_company_overview_data(Company_Overview):
    """"gets the company overview data (dont the name of the func speaks for itself?)"""
    Company_Overview_dict = {"size":"",
                            "founded":"",
                            "type":"",
                            "industry":"",
                            "sector":"",
                            "revenue":"",}

    for i, key in enumerate(Company_Overview_dict):
        try: 
            Company_Overview_dict[key] = Company_Overview[i].text
        except:
            print(f"{key} not found")
            Company_Overview_dict[key] = -1
    return Company_Overview_dict




### SCRAPPER

In [3]:
path = "/opt/homebrew/bin/chromedriver"
keyword = "data science"
num_jobs = 15
slp_time = 5
verbose = True



def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    """Gathers jobs as a dataframe, scraped from Glassdoor"""
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    #driver.set_window_size(1120, 1000)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []
    time.sleep(slp_time)
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        
        #Going through each job in this page
        job_buttons = driver.find_elements(By.CLASS_NAME, "react-job-listing")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print(f"Progress:  {str(len(jobs))}/{ str(num_jobs)}")
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(slp_time)
            #close modal (just appear on the first pass)
            if len(jobs) == 0:
                try:
                    driver.find_element(By.XPATH, '//*[@id="JAModal"]/div/div[2]/span').click()
                except NoSuchElementException:
                    print("no hay modal xd")
            collected_successfully = False
            while not collected_successfully:
                try:
                    company_name_ = driver.find_element(By.CLASS_NAME, 'e1tk4kwz5').text
                    #sometimes company name comes with the rating, if its the case get_companyname_rating handles it
                    company_name, rating = get_companyname_rating(company_name_) 
                    location = driver.find_element(By.CLASS_NAME,'e1tk4kwz1').text
                    job_title = driver.find_element(By.CLASS_NAME,'e1tk4kwz2').text
                    job_description = driver.find_element(By.XPATH,'.//div[@class="jobDescriptionContent desc"]').text
                    salary_estimate = driver.find_element(By.CLASS_NAME,'e2u4hf18').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            job_data_dict = {"Job Title" : job_title,
                            "Company Name" : company_name,
                            "Location" : location,
                            "Salary Estimate" : salary_estimate,
                            "Rating" : rating,
                            "Job Description" : job_description,
                        }

            #Printing for debugging
            if verbose:
                print(job_data_dict)

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                Company_Overview = driver.find_elements(By.CLASS_NAME,'e1pvx6aw0')
                company_overview_dict = get_company_overview_data(Company_Overview)

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                company_overview_dict = {"size":-1,
                                        "founded":-1,
                                        "type":-1,
                                        "industry":-1,
                                        "sector":-1,
                                        "revenue":-1}
                
            if verbose:
                print(company_overview_dict)
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
            
            job_data_dict.update(company_overview_dict)

            jobs.append(job_data_dict)

            #add job to jobs
            
            
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
        
    driver.close()
    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [4]:

path = "/opt/homebrew/bin/chromedriver"
keyword = "data science"
num_jobs = 15
slp_time = 5
verbose = True



df = get_jobs(keyword, num_jobs, verbose, path, slp_time)
df

/var/folders/n2/np9y87ks47j0gqm4v82gvddh0000gn/T/ipykernel_57812/780972641.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)


Progress:  0/15
{'Job Title': 'Associate, Data Science (Remote)', 'Company Name': 'Lincoln Financial', 'Location': 'Radnor, PA', 'Salary Estimate': '$74,800 /yr (est.)', 'Rating': '3.6', 'Job Description': "Alternate Locations: Work from Home\n\nWork Arrangement: Work from Home\n\nRelocation assistance:: is not available for this opportunity.\n\nPay Range: $65,900 - $83,700\n\nBonus Potential:\n\nRequisition #: 68905\n\nThe Role at a Glance:\nWe're excited to add an Associate of Data Science to our team. In this position you will focus on developing advanced analytical work products to facilitate the success of Lincoln’s distribution organization (LFD). The employee will work closely with LFD’s business lines, marketing and IT teams to create novel solutions (e.g. models, processes and tools) that address the everchanging needs of a dynamic sales organization. The candidate must exhibit a firm grasp of the concepts and skills within the Data Science discipline as well as the technical 